In [ ]:
# Configuración de cores seguros para datasets grandes
import os
N_JOBS = 16  # Reducido drásticamente para evitar memory overflow
os.environ['OMP_NUM_THREADS'] = str(N_JOBS)
os.environ['MKL_NUM_THREADS'] = str(N_JOBS)
os.environ['OPENBLAS_NUM_THREADS'] = str(N_JOBS)
print(f"Usando {N_JOBS} cores (configuración segura para datasets grandes)")

Usando 16 cores (configuración segura para datasets grandes)


In [2]:
!pip install scikit-learn-intelex xgboost
from sklearnex import patch_sklearn
patch_sklearn()
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import numpy as np

Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

import time


In [ ]:
df_green = pd.read_parquet('df_all_green_dn_clean_e3.parquet')

In [5]:
df_green

,vendor_name,passenger_count,rate_code_desc,total_amount,pickup_dow,pickup_month,pickup_year,pickup_hour,is_weekend,is_rush_hour,pickup_borough
0,"Curb Mobility, LLC",1.0,Standard rate,14.30,6,1,2016,0,False,False,Manhattan
1,"Curb Mobility, LLC",3.0,Standard rate,11.80,6,1,2016,0,False,False,Brooklyn
2,"Curb Mobility, LLC",1.0,Standard rate,13.80,6,1,2016,0,False,False,Manhattan
3,"Curb Mobility, LLC",4.0,Standard rate,6.96,6,1,2016,0,False,False,Manhattan
4,"Curb Mobility, LLC",1.0,Standard rate,15.36,6,1,2016,0,False,False,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...
878559,"Curb Mobility, LLC",1.0,Standard rate,30.80,3,12,2019,23,False,False,Manhattan
878560,"Curb Mobility, LLC",1.0,Standard rate,11.80,3,12,2019,23,False,False,Manhattan
878561,"Creative Mobile Technologies, LLC",1.0,Standard rate,7.80,3,12,2019,23,False,False,Manhattan
878562,"Creative Mobile Technologies, LLC",2.0,Standard rate,12.30,3,12,2019,23,False,False,Manhattan


In [6]:
df_green.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878564 entries, 0 to 878563
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   vendor_name      878564 non-null  object 
 1   passenger_count  878564 non-null  float64
 2   rate_code_desc   878564 non-null  object 
 3   total_amount     878564 non-null  float64
 4   pickup_dow       878564 non-null  int32  
 5   pickup_month     878564 non-null  int32  
 6   pickup_year      878564 non-null  int32  
 7   pickup_hour      878564 non-null  int32  
 8   is_weekend       878564 non-null  bool   
 9   is_rush_hour     878564 non-null  bool   
 10  pickup_borough   878564 non-null  object 
dtypes: bool(2), float64(2), int32(4), object(3)
memory usage: 48.6+ MB


In [7]:
df_train = df_green[df_green["pickup_year"].isin([2016, 2017])]
df_val = df_green[df_green["pickup_year"] == 2018]
df_test = df_green[df_green["pickup_year"] == 2019]
print(f"Train shape: {df_train.shape}, Val shape: {df_val.shape}, Test shape: {df_test.shape}")


target = "total_amount"
feature_cols = [col for col in df_green.columns if col != target]

X_train = df_train.drop(columns=[target])
y_train = df_train[target]

X_val = df_val.drop(columns=[target])
y_val = df_val[target]

X_test = df_test.drop(columns=[target])
y_test = df_test[target]


Train shape: (421532, 11), Val shape: (173813, 11), Test shape: (283219, 11)


## Baseline

In [8]:
# MEDIA
y_train_mean = y_train.mean()

# Predecir la media para validación y test
y_val_pred_baseline = np.full(len(y_val), y_train_mean)
y_test_pred_baseline = np.full(len(y_test), y_train_mean)

# Calcular métricas
# Validación
val_rmse_baseline = np.sqrt(mean_squared_error(y_val, y_val_pred_baseline))
val_mae_baseline = mean_absolute_error(y_val, y_val_pred_baseline)

# Test  
test_rmse_baseline = np.sqrt(mean_squared_error(y_test, y_test_pred_baseline))
test_mae_baseline = mean_absolute_error(y_test, y_test_pred_baseline)

print(f"Baseline (Media):")
print(f"Validación - RMSE: {val_rmse_baseline:.4f}, MAE: {val_mae_baseline:.4f}")
print(f"Test - RMSE: {test_rmse_baseline:.4f}, MAE: {test_mae_baseline:.4f}")



Baseline (Media):
Validación - RMSE: 6.2441, MAE: 4.7908
Test - RMSE: 6.7068, MAE: 4.7513


## Ciclicas

In [9]:
df_green_d= df_green.copy()

In [10]:
import numpy as np
import pandas as pd

# Asume que df_train es tu DataFrame de entrenamiento (ya en Pandas)
# Y que las columnas 'pickup_hour', 'pickup_dow', 'pickup_month' son INTEGERS.

# --- 1. Variable pickup_hour (Periodo = 24 horas) ---
# Creamos dos nuevas columnas:
df_green_d['hour_sin'] = np.sin(2 * np.pi * df_green_d['pickup_hour'] / 24)
df_green_d['hour_cos'] = np.cos(2 * np.pi * df_green_d['pickup_hour'] / 24)

# --- 2. Variable pickup_dow (Día de la Semana, Periodo = 7 días) ---
# Asumimos que la columna 'pickup_dow' tiene valores entre 1 y 7.
df_green_d['dow_sin'] = np.sin(2 * np.pi * df_green_d['pickup_dow'] / 7)
df_green_d['dow_cos'] = np.cos(2 * np.pi * df_green_d['pickup_dow'] / 7)

# --- 3. Variable pickup_month (Mes del Año, Periodo = 12 meses) ---
# Asumimos que la columna 'pickup_month' tiene valores entre 1 y 12.
df_green_d['month_sin'] = np.sin(2 * np.pi * df_green_d['pickup_month'] / 12)
df_green_d['month_cos'] = np.cos(2 * np.pi * df_green_d['pickup_month'] / 12)

# Opcional: Eliminar las columnas originales después de la transformación
df_green_d_t = df_green_d.drop(columns=['pickup_hour', 'pickup_dow', 'pickup_month'])


In [11]:
df_green_d_t

,vendor_name,passenger_count,rate_code_desc,total_amount,pickup_year,is_weekend,is_rush_hour,pickup_borough,hour_sin,hour_cos,dow_sin,dow_cos,month_sin,month_cos
0,"Curb Mobility, LLC",1.0,Standard rate,14.30,2016,False,False,Manhattan,0.000000,1.000000,-0.781831,0.623490,5.000000e-01,0.866025
1,"Curb Mobility, LLC",3.0,Standard rate,11.80,2016,False,False,Brooklyn,0.000000,1.000000,-0.781831,0.623490,5.000000e-01,0.866025
2,"Curb Mobility, LLC",1.0,Standard rate,13.80,2016,False,False,Manhattan,0.000000,1.000000,-0.781831,0.623490,5.000000e-01,0.866025
3,"Curb Mobility, LLC",4.0,Standard rate,6.96,2016,False,False,Manhattan,0.000000,1.000000,-0.781831,0.623490,5.000000e-01,0.866025
4,"Curb Mobility, LLC",1.0,Standard rate,15.36,2016,False,False,Manhattan,0.000000,1.000000,-0.781831,0.623490,5.000000e-01,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878559,"Curb Mobility, LLC",1.0,Standard rate,30.80,2019,False,False,Manhattan,-0.258819,0.965926,0.433884,-0.900969,-2.449294e-16,1.000000
878560,"Curb Mobility, LLC",1.0,Standard rate,11.80,2019,False,False,Manhattan,-0.258819,0.965926,0.433884,-0.900969,-2.449294e-16,1.000000
878561,"Creative Mobile Technologies, LLC",1.0,Standard rate,7.80,2019,False,False,Manhattan,-0.258819,0.965926,0.433884,-0.900969,-2.449294e-16,1.000000
878562,"Creative Mobile Technologies, LLC",2.0,Standard rate,12.30,2019,False,False,Manhattan,-0.258819,0.965926,0.433884,-0.900969,-2.449294e-16,1.000000


In [12]:
df_train_t = df_green_d_t[df_green_d_t["pickup_year"].isin([2016, 2017])]
df_val_t = df_green_d_t[df_green_d_t["pickup_year"] == 2018]
df_test_t = df_green_d_t[df_green_d_t["pickup_year"] == 2019]
print(f"Train shape: {df_train_t.shape}, Val shape: {df_val_t.shape}, Test shape: {df_test_t.shape}")

target = "total_amount"
feature_cols = [col for col in df_green.columns if col != target]

X_train_t = df_train_t.drop(columns=[target])
y_train_t = df_train_t[target]

X_val_t = df_val_t.drop(columns=[target])
y_val_t = df_val_t[target]

X_test_t = df_test_t.drop(columns=[target])
y_test_t = df_test_t[target]


Train shape: (421532, 14), Val shape: (173813, 14), Test shape: (283219, 14)


In [13]:
categorical_discrete = ['pickup_borough', 'vendor_name', 'rate_code_desc'] #OHE
numeric_features = ['passenger_count','is_weekend','is_rush_hour','hour_sin','hour_cos','dow_sin','dow_cos','month_sin','month_cos'] 


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features), 
        ('cat', OneHotEncoder(), categorical_discrete)
    ],
    remainder='drop' 
)

# Escalar usando solo datos de entrenamiento
X_train_scaled_t= X_train_t.copy()
X_val_scaled_t = X_val_t.copy()
X_test_scaled_t = X_test_t.copy()

X_train_scaled_t = preprocessor.fit_transform(X_train_t)
X_val_scaled_t = preprocessor.transform(X_val_t)
X_test_scaled_t = preprocessor.transform(X_test_t)


In [14]:
  # Combinar train + val
X_combined_t = np.vstack([X_train_scaled_t, X_val_scaled_t])
y_combined_t = np.hstack([y_train_t, y_val_t])


## Voting


In [15]:
#PROBAR DIFERENTES REGRESORES 
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

poly_pipeline = Pipeline([
      ('poly', PolynomialFeatures(degree=2, include_bias=False)),
      ('linear', LinearRegression())
  ])

# 1. Definir 3 regresores base diferentes
estimators = [
    ('poly', poly_pipeline),
    ('forest', RandomForestRegressor(random_state=42)),
    ('ridge', Ridge())
]

# 2. Hard Voting (promedio simple)
voting_hard = VotingRegressor(
    estimators=estimators,
    n_jobs=N_JOBS
  )


# Hiperparámetros para cada modelo base
param_grid_voting = {
    'poly__poly__degree': [2, 3],
    # RandomForest parameters (prefijo: forest__)
    'forest__n_estimators': [50, 100],
    'forest__max_depth': [10, 20],
    'forest__min_samples_split': [2, 5],

    # SVR parameters (prefijo: svm__)
    'ridge__alpha': [0.1, 1.0, 10.0],
  }

train_idx_t = np.arange(len(X_train_t))                     
val_idx_t = np.arange(len(X_train_t), len(X_combined_t))

  # GridSearch
grid_voting_t = GridSearchCV(
    estimator=voting_hard,
    param_grid=param_grid_voting,
    cv = [(train_idx_t, val_idx_t)],  
    scoring='neg_mean_squared_error', 
    n_jobs=N_JOBS,
    verbose=1
)

In [16]:
# #PROBAR DIFERENTES REGRESORES 
# from sklearn.ensemble import VotingRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import GridSearchCV

# # 1. Definir 3 regresores base diferentes
# estimators = [
#     ('linear', LinearRegression()),
#     ('forest', RandomForestRegressor(random_state=42)),
#     ('ridge', Ridge())
# ]

# # 2. Hard Voting (promedio simple)
# voting_hard = VotingRegressor(
#     estimators=estimators,
#     n_jobs=N_JOBS
#   )


# # Hiperparámetros para cada modelo base
# param_grid_voting = {

#     # RandomForest parameters (prefijo: forest__)
#     'forest__n_estimators': [50, 100],
#     'forest__max_depth': [10, 20],
#     'forest__min_samples_split': [2, 5],

#     # SVR parameters (prefijo: svm__)
#     'ridge__alpha': [0.1, 1.0, 10.0],
#   }

# train_idx_t = np.arange(len(X_train_t))                     
# val_idx_t = np.arange(len(X_train_t), len(X_combined_t))

#   # GridSearch
# grid_voting_t = GridSearchCV(
#     estimator=voting_hard,
#     param_grid=param_grid_voting,
#     cv = [(train_idx_t, val_idx_t)],  
#     scoring='neg_mean_squared_error', 
#     n_jobs=N_JOBS,
#     verbose=1
# )


In [17]:
  # Entrenar Registrar tiempos de entrenamiento e inferencia.
grid_voting_t.fit(X_combined_t, y_combined_t)

Fitting 1 folds for each of 48 candidates, totalling 48 fits


/home/staffteam/.venv/lib/python3.10/site-packages/daal4py/sklearn/_n_jobs_support.py:83: UserWarning: 'Threading' parallel backend is not supported by Extension for Scikit-learn*. Falling back to usage of all available threads.
  warn(
/home/staffteam/.venv/lib/python3.10/site-packages/daal4py/sklearn/_n_jobs_support.py:83: UserWarning: 'Threading' parallel backend is not supported by Extension for Scikit-learn*. Falling back to usage of all available threads.
  warn(
/home/staffteam/.venv/lib/python3.10/site-packages/daal4py/sklearn/_n_jobs_support.py:83: UserWarning: 'Threading' parallel backend is not supported by Extension for Scikit-learn*. Falling back to usage of all available threads.
  warn(
/home/staffteam/.venv/lib/python3.10/site-packages/daal4py/sklearn/_n_jobs_support.py:83: UserWarning: 'Threading' parallel backend is not supported by Extension for Scikit-learn*. Falling back to usage of all available threads.
  warn(
/home/staffteam/.venv/lib/python3.10/site-packages/d

,estimator,VotingRegress... n_jobs=16)
,param_grid,"{'forest__max_depth': [10, 20], 'forest__min_samples_split': [2, 5], 'forest__n_estimators': [50, 100], 'poly__poly__degree': [2, 3], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,16
,refit,True
,cv,"[(array([ 0...ape=(421532,)), ...)]"
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,degree,2


In [18]:
  # Mejores parámetros
print("Mejores parámetros:", grid_voting_t.best_params_)

  # Predicciones
y_val_pred_voting_t = grid_voting_t.predict(X_val_scaled_t)
y_test_pred_voting_t = grid_voting_t.predict(X_test_scaled_t)

  # Métricas
val_rmse_t = np.sqrt(mean_squared_error(y_val_t, y_val_pred_voting_t))
val_mae_t = mean_absolute_error(y_val_t, y_val_pred_voting_t)
val_r2_t= r2_score(y_val_t, y_val_pred_voting_t)

test_rmse_t = np.sqrt(mean_squared_error(y_test_t, y_test_pred_voting_t))
test_mae_t = mean_absolute_error(y_test_t, y_test_pred_voting_t)
test_r2_t= r2_score(y_test_t, y_test_pred_voting_t)

print(f"Voting - Validación RMSE: {val_rmse_t:.4f}, MAE: {val_mae_t:.4f}, R2: {val_r2_t:.4f}")
print(f"Voting - Validación RMSE: {test_rmse_t:.4f}, MAE: {test_mae_t:.4f}, R2: {test_r2_t:.4f}")


Mejores parámetros: {'forest__max_depth': 10, 'forest__min_samples_split': 2, 'forest__n_estimators': 50, 'poly__poly__degree': 2, 'ridge__alpha': 0.1}
Voting - Validación RMSE: 6.1080, MAE: 4.6916, R2: 0.0430
Voting - Validación RMSE: 6.6376, MAE: 4.7112, R2: -0.1243


In [19]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

base_tree = DecisionTreeRegressor(random_state=42)

  # BAGGING 
bagging_model_t = BaggingRegressor(
    estimator=base_tree,
    n_estimators=100,
    bootstrap=True,        
    random_state=42,
    n_jobs=4  # Limitado para bagging interno
  )

# PASTING  
pasting_model_t = BaggingRegressor(
    estimator=base_tree,
    n_estimators=100,
    bootstrap=False,       
    random_state=42,
    n_jobs=4  # Limitado para pasting interno
)



  # Parámetros REDUCIDOS para evitar memory overflow
param_grid_bagging = {
    'n_estimators': [50, 100],                    # Reducido de 3 a 2 opciones
    'max_samples': [0.7, 1.0],                    # Reducido de 3 a 2 opciones
    'max_features': [0.7, 1.0],                   # Reducido de 3 a 2 opciones
    'estimator__max_depth': [5, 10, None],        # Reducido de 4 a 3 opciones
    'estimator__min_samples_split': [2, 10]       # Reducido de 3 a 2 opciones
}
# Total: 2*2*2*3*2 = 48 combinaciones (vs 324 anteriores)

grid_bagging_t = GridSearchCV(
    estimator=bagging_model_t,
    param_grid=param_grid_bagging,
    cv=[(train_idx_t, val_idx_t)],
    scoring='neg_mean_squared_error',
    n_jobs=N_JOBS,  # Paralelización en GridSearch
    verbose=2,      # Más información para debugging
    pre_dispatch='2*n_jobs'  # Control de memoria
)


grid_pasting_t = GridSearchCV(
    estimator=pasting_model_t,
    param_grid=param_grid_bagging, 
    cv=[(train_idx_t, val_idx_t)],
    scoring='neg_mean_squared_error',
    n_jobs=N_JOBS,
    verbose=2,
    pre_dispatch='2*n_jobs'
)

In [20]:
grid_bagging_t.fit(X_combined_t, y_combined_t)


Fitting 1 folds for each of 48 candidates, totalling 48 fits
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=1.0, n_estimators=50; total time=   6.6s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=50; total time=   6.6s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=0.7, n_estimators=50; total time=   6.6s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=100; total time=   8.8s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=1.0, n_estimators=50; total time=   7.5s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=1.0, max_samples=0.7, n_estimators=50; total time=  10.7s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=1.0, n_estimators=100; to

/home/staffteam/.venv/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=1.0, max_samples=0.7, n_estimators=50; total time=  11.3s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=0.7, n_estimators=100; total time=  11.9s
[CV] END estimator__max_depth=10, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=50; total time=   8.3s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=1.0, n_estimators=100; total time=  14.8s
[CV] END estimator__max_depth=10, estimator__min_samples_split=2, max_features=0.7, max_samples=1.0, n_estimators=50; total time=   8.8s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=1.0, max_samples=0.7, n_estimators=100; total time=  19.4s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=1.0, max_samples=1.0, n_estimators=100; total time=  20.0s
[CV] END estimator__max_depth=10, esti

,estimator,BaggingRegres...ndom_state=42)
,param_grid,"{'estimator__max_depth': [5, 10, ...], 'estimator__min_samples_split': [2, 10], 'max_features': [0.7, 1.0], 'max_samples': [0.7, 1.0], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,16
,refit,True
,cv,"[(array([ 0...ape=(421532,)), ...)]"
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [21]:
grid_pasting_t.fit(X_combined_t, y_combined_t)

Fitting 1 folds for each of 48 candidates, totalling 48 fits


[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=0.7, n_estimators=50; total time=   7.5s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=50; total time=   7.6s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=0.7, max_samples=1.0, n_estimators=50; total time=   8.5s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=1.0, n_estimators=50; total time=   8.7s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=1.0, max_samples=0.7, n_estimators=50; total time=  11.9s
[CV] END estimator__max_depth=5, estimator__min_samples_split=2, max_features=0.7, max_samples=0.7, n_estimators=100; total time=  14.3s
[CV] END estimator__max_depth=5, estimator__min_samples_split=10, max_features=1.0, max_samples=0.7, n_estimators=50; total time=  15.1s
[CV] END estimator__max_depth=5, estimator__

,estimator,BaggingRegres...ndom_state=42)
,param_grid,"{'estimator__max_depth': [5, 10, ...], 'estimator__min_samples_split': [2, 10], 'max_features': [0.7, 1.0], 'max_samples': [0.7, 1.0], ...}"
,scoring,'neg_mean_squared_error'
,n_jobs,16
,refit,True
,cv,"[(array([ 0...ape=(421532,)), ...)]"
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [22]:
  # Mejores parámetros BAGGING
print("Mejores parámetros:", grid_bagging_t.best_params_)

  # Predicciones
y_val_pred_bagging_t = grid_bagging_t.predict(X_val_scaled_t)
y_test_pred_bagging_t = grid_bagging_t.predict(X_test_scaled_t)

  # Métricas
val_rmse_bagging_t = np.sqrt(mean_squared_error(y_val_t, y_val_pred_bagging_t))
val_mae_bagging_t = mean_absolute_error(y_val_t, y_val_pred_bagging_t)
val_r2_bagging_t= r2_score(y_val_t, y_val_pred_bagging_t)

test_rmse_bagging_t = np.sqrt(mean_squared_error(y_test_t, y_test_pred_bagging_t))
test_mae_bagging_t = mean_absolute_error(y_test_t, y_test_pred_bagging_t)
test_r2_bagging_t= r2_score(y_test_t, y_test_pred_bagging_t)

print(f"Bagging - Validación RMSE: {val_rmse_bagging_t:.4f}, MAE: {val_mae_bagging_t:.4f}, R2: {val_r2_bagging_t:.4f}")
print(f"Bagging - Validación RMSE: {test_rmse_bagging_t:.4f}, MAE: {test_mae_bagging_t:.4f}, R2: {test_r2_bagging_t:.4f}")

Mejores parámetros: {'estimator__max_depth': 10, 'estimator__min_samples_split': 10, 'max_features': 1.0, 'max_samples': 0.7, 'n_estimators': 100}
Bagging - Validación RMSE: 6.0877, MAE: 4.6759, R2: 0.0494
Bagging - Validación RMSE: 6.6350, MAE: 4.7096, R2: -0.1234


In [23]:
  # Mejores parámetros PASTING
print("Mejores parámetros:", grid_pasting_t.best_params_)

  # Predicciones
y_val_pred_pasting_t = grid_pasting_t.predict(X_val_scaled_t)
y_test_pred_pasting_t = grid_pasting_t.predict(X_test_scaled_t)

  # Métricas
val_rmse_pasting_t = np.sqrt(mean_squared_error(y_val_t, y_val_pred_pasting_t))
val_mae_pasting_t = mean_absolute_error(y_val_t, y_val_pred_pasting_t)
val_r2_pasting_t= r2_score(y_val_t, y_val_pred_pasting_t)

test_rmse_pasting_t = np.sqrt(mean_squared_error(y_test_t, y_test_pred_pasting_t))
test_mae_pasting_t = mean_absolute_error(y_test_t, y_test_pred_pasting_t)
test_r2_pasting_t= r2_score(y_test_t, y_test_pred_pasting_t)

print(f"pasting - Validación RMSE: {val_rmse_pasting_t:.4f}, MAE: {val_mae_pasting_t:.4f}, R2: {val_r2_pasting_t:.4f}")
print(f"pasting - Validación RMSE: {test_rmse_pasting_t:.4f}, MAE: {test_mae_pasting_t:.4f}, R2: {test_r2_pasting_t:.4f}")

Mejores parámetros: {'estimator__max_depth': 10, 'estimator__min_samples_split': 10, 'max_features': 1.0, 'max_samples': 0.7, 'n_estimators': 100}
pasting - Validación RMSE: 6.0868, MAE: 4.6748, R2: 0.0497
pasting - Validación RMSE: 6.6367, MAE: 4.7105, R2: -0.1240


In [24]:
#OHE
#categorical= ['pickup_borough', 'vendor_name', 'rate_code_desc','pickup_hour','pickup_dow','pickup_month','is_weekend','is_rush_hour'] #OHE
categorical= ['pickup_borough', 'vendor_name', 'rate_code_desc','is_weekend','is_rush_hour']
numeric_features = ['passenger_count'] 

#Dtypes de categorías: {'vendor_name': dtype('O'), 'rate_code_desc': dtype('O'), 'pickup_borough': dtype('O'), 'is_weekend': dtype('O'), 'is_rush_hour': dtype('O')}

In [25]:
# Para CatBoost necesitamos datos SIN codificar con categorías como strings
X_train_catboost = X_train.copy()
X_val_catboost = X_val.copy()

# Convertir columnas categóricas a string
for col in categorical:
    X_train_catboost[col] = X_train_catboost[col].astype(str)
    X_val_catboost[col] = X_val_catboost[col].astype(str)

X_combined_catboost = pd.concat([X_train_catboost, X_val_catboost], axis=0, ignore_index=True)

# Obtener índices de columnas categóricas
cat_indices = [X_train_catboost.columns.get_loc(col) for col in categorical]
print(f"Índices de columnas categóricas: {cat_indices}")
print(f"Shape X_combined_catboost: {X_combined_catboost.shape}")
print(f"Dtypes de categorías: {X_combined_catboost[categorical].dtypes.to_dict()}")

Índices de columnas categóricas: [9, 0, 2, 7, 8]
Shape X_combined_catboost: (595345, 10)
Dtypes de categorías: {'pickup_borough': dtype('O'), 'vendor_name': dtype('O'), 'rate_code_desc': dtype('O'), 'is_weekend': dtype('O'), 'is_rush_hour': dtype('O')}


In [26]:
#CatBoost

from catboost import CatBoostRegressor

catboost_model = CatBoostRegressor(
    random_seed=42,
    verbose=False,
    task_type='GPU',
    devices='0',
    cat_features=cat_indices,  # USAR ÍNDICES en lugar de nombres
    thread_count=N_JOBS  # Controlar threads en GPU
)

  # Parámetros REDUCIDOS para CatBoost
param_grid_catboost = {
    'iterations': [100, 300],              # Reducido de 3 a 2 opciones
    'learning_rate': [0.01, 0.1],          # Reducido de 3 a 2 opciones
    'depth': [4, 6, 8],                    # Mantener 3 opciones
    'l2_leaf_reg': [1, 3],                 # Reducido de 3 a 2 opciones
}
# Total: 2*2*3*2 = 24 combinaciones (vs 243 anteriores)

grid_catboost = GridSearchCV(
    estimator=catboost_model,
    param_grid=param_grid_catboost,
    cv=[(np.arange(len(X_train)), np.arange(len(X_val)))],
    scoring='neg_mean_squared_error',
    n_jobs=1,  # CatBoost ya usa paralelización interna con GPU
    verbose=2
)

In [27]:
# Usar datos SIN codificar para CatBoost
# CatBoost: depth, learning_rate, l2_leaf_reg, bagging_temperature,
# manejo nativo de categóricas; loss_function de regresión.
# Para cada uno: Grid/Random Search + CV temporal; registrar mejor set y
# sensibilidad.
grid_catboost.fit(X_combined_catboost, y_combined_t)

Fitting 1 folds for each of 24 candidates, totalling 24 fits
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   1.8s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.1; total time=   2.6s
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.01; total time=   4.1s
[CV] END depth=4, iterations=100, l2_leaf_reg=3, learning_rate=0.1; total time=   2.3s
[CV] END depth=4, iterations=300, l2_leaf_reg=1, learning_rate=0.01; total time=   3.9s
[CV] END depth=4, iterations=300, l2_leaf_reg=1, learning_rate=0.1; total time=   3.8s
[CV] END depth=4, iterations=300, l2_leaf_reg=3, learning_rate=0.01; total time=   4.0s
[CV] END depth=4, iterations=300, l2_leaf_reg=3, learning_rate=0.1; total time=   3.9s
[CV] END depth=6, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=   2.6s
[CV] END depth=6, iterations=100, l2_leaf_reg=1, learning_rate=0.1; total time=   2.7s
[CV] END depth=6, iterations=100, l2_leaf_reg=3, learning_rate=0

,estimator,<catboost.cor...x7f50a8807eb0>
,param_grid,"{'depth': [4, 6, ...], 'iterations': [100, 300], 'l2_leaf_reg': [1, 3], 'learning_rate': [0.01, 0.1]}"
,scoring,'neg_mean_squared_error'
,n_jobs,1
,refit,True
,cv,"[(array([ 0...ape=(421532,)), ...)]"
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False


In [28]:
# # Resumen completo de todos los modelos entrenados

# print("="*80)
# print("RESUMEN DE RESULTADOS - MODELOS DE PREDICCIÓN DE TOTAL_AMOUNT")
# print("="*80)
# print(f"\nDATASET:")
# print(f"  - Train: {df_train.shape[0]:,} muestras")
# print(f"  - Validación: {df_val.shape[0]:,} muestras")
# print(f"  - Test: {df_test.shape[0]:,} muestras")
# print(f"  - Features: {len(feature_cols)}")
# print(f"  - Cores utilizados: {N_JOBS}")

# print("\n" + "="*80)
# print("1. VOTING REGRESSOR")
# print("="*80)
# print(f"Mejores parámetros: {grid_voting.best_params_}")
# print(f"Mejor score (neg MSE): {grid_voting.best_score_:.4f}")
# print(f"\nMétricas en Validación:")
# print(f"  - RMSE: {val_rmse:.4f}")
# print(f"  - MAE:  {val_mae:.4f}")
# print(f"  - R²:   {val_r2:.4f}")

# print("\n" + "="*80)
# print("2. BAGGING REGRESSOR")
# print("="*80)
# print(f"Mejores parámetros: {grid_bagging.best_params_}")
# print(f"Mejor score (neg MSE): {grid_bagging.best_score_:.4f}")
# bagging_rmse = np.sqrt(-grid_bagging.best_score_)
# print(f"RMSE estimado: {bagging_rmse:.4f}")

# print("\n" + "="*80)
# print("3. PASTING REGRESSOR")
# print("="*80)
# print(f"Mejores parámetros: {grid_pasting.best_params_}")
# print(f"Mejor score (neg MSE): {grid_pasting.best_score_:.4f}")
# pasting_rmse = np.sqrt(-grid_pasting.best_score_)
# print(f"RMSE estimado: {pasting_rmse:.4f}")

# print("\n" + "="*80)
# print("4. CATBOOST REGRESSOR")
# print("="*80)
# print(f"Mejores parámetros: {grid_catboost.best_params_}")
# print(f"Mejor score (neg MSE): {grid_catboost.best_score_:.4f}")
# catboost_rmse = np.sqrt(-grid_catboost.best_score_)
# print(f"RMSE estimado: {catboost_rmse:.4f}")

# print("\n" + "="*80)
# print("COMPARACIÓN FINAL")
# print("="*80)
# results_summary = pd.DataFrame({
#     'Modelo': ['Voting', 'Bagging', 'Pasting', 'CatBoost'],
#     'RMSE': [val_rmse, bagging_rmse, pasting_rmse, catboost_rmse],
#     'Neg MSE Score': [grid_voting.best_score_, grid_bagging.best_score_, 
#                       grid_pasting.best_score_, grid_catboost.best_score_]
# })
# results_summary = results_summary.sort_values('RMSE')
# print(results_summary.to_string(index=False))

# print("\n" + "="*80)
# best_model = results_summary.iloc[0]['Modelo']
# best_rmse = results_summary.iloc[0]['RMSE']
# print(f"MEJOR MODELO: {best_model} con RMSE = {best_rmse:.4f}")
# print("="*80)

In [29]:
# Resumen completo de todos los modelos entrenados

print("="*80)
print("RESUMEN DE RESULTADOS - MODELOS DE PREDICCIÓN DE TOTAL_AMOUNT")
print("="*80)
print(f"\nDATASET:")
print(f"  - Train: {df_train.shape[0]:,} muestras")
print(f"  - Validación: {df_val.shape[0]:,} muestras")
print(f"  - Test: {df_test.shape[0]:,} muestras")
print(f"  - Features: {len(feature_cols)}")
print(f"  - Cores utilizados: {N_JOBS}")

print("\n" + "="*80)
print("1. VOTING REGRESSOR")
print("="*80)
print(f"Mejores parámetros: {grid_voting_t.best_params_}")
print(f"Mejor score (neg MSE): {grid_voting_t.best_score_:.4f}")
print(f"\nMétricas en Validación:")
print(f"  - RMSE: {val_rmse_t:.4f}")
print(f"  - MAE:  {val_mae_t:.4f}")
print(f"  - R²:   {val_r2_t:.4f}")

print("\n" + "="*80)
print("2. BAGGING REGRESSOR")
print("="*80)
print(f"Mejores parámetros: {grid_bagging_t.best_params_}")
print(f"Mejor score (neg MSE): {grid_bagging_t.best_score_:.4f}")
bagging_rmse = np.sqrt(-grid_bagging_t.best_score_)
print(f"RMSE estimado: {bagging_rmse:.4f}")

print("\n" + "="*80)
print("3. PASTING REGRESSOR")
print("="*80)
print(f"Mejores parámetros: {grid_pasting_t.best_params_}")
print(f"Mejor score (neg MSE): {grid_pasting_t.best_score_:.4f}")
pasting_rmse = np.sqrt(-grid_pasting_t.best_score_)
print(f"RMSE estimado: {pasting_rmse:.4f}")

print("\n" + "="*80)
print("4. CATBOOST REGRESSOR")
print("="*80)
print(f"Mejores parámetros: {grid_catboost.best_params_}")
print(f"Mejor score (neg MSE): {grid_catboost.best_score_:.4f}")
catboost_rmse = np.sqrt(-grid_catboost.best_score_)
print(f"RMSE estimado: {catboost_rmse:.4f}")

print("\n" + "="*80)
print("COMPARACIÓN FINAL")
print("="*80)
results_summary = pd.DataFrame({
    'Modelo': ['Voting', 'Bagging', 'Pasting', 'CatBoost'],
    'RMSE': [val_rmse_t, bagging_rmse, pasting_rmse, catboost_rmse],
    'Neg MSE Score': [grid_voting_t.best_score_, grid_bagging_t.best_score_, 
                      grid_pasting_t.best_score_, grid_catboost.best_score_]
})
results_summary = results_summary.sort_values('RMSE')
print(results_summary.to_string(index=False))

print("\n" + "="*80)
best_model = results_summary.iloc[0]['Modelo']
best_rmse = results_summary.iloc[0]['RMSE']
print(f"MEJOR MODELO: {best_model} con RMSE = {best_rmse:.4f}")
print("="*80)

RESUMEN DE RESULTADOS - MODELOS DE PREDICCIÓN DE TOTAL_AMOUNT

DATASET:
  - Train: 421,532 muestras
  - Validación: 173,813 muestras
  - Test: 283,219 muestras
  - Features: 10
  - Cores utilizados: 16

1. VOTING REGRESSOR
Mejores parámetros: {'forest__max_depth': 10, 'forest__min_samples_split': 2, 'forest__n_estimators': 50, 'poly__poly__degree': 2, 'ridge__alpha': 0.1}
Mejor score (neg MSE): -37.4975

Métricas en Validación:
  - RMSE: 6.1080
  - MAE:  4.6916
  - R²:   0.0430

2. BAGGING REGRESSOR
Mejores parámetros: {'estimator__max_depth': 10, 'estimator__min_samples_split': 10, 'max_features': 1.0, 'max_samples': 0.7, 'n_estimators': 100}
Mejor score (neg MSE): -37.4687
RMSE estimado: 6.1212

3. PASTING REGRESSOR
Mejores parámetros: {'estimator__max_depth': 10, 'estimator__min_samples_split': 10, 'max_features': 1.0, 'max_samples': 0.7, 'n_estimators': 100}
Mejor score (neg MSE): -37.4893
RMSE estimado: 6.1228

4. CATBOOST REGRESSOR
Mejores parámetros: {'depth': 8, 'iterations': 3